In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yatinjamwal","key":"92a3eac589b266760fb5d03a214c8da8"}'}

In [3]:
!mkdir ~/.kaggle

In [4]:
!cp kaggle.json ~/.kaggle/

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets list

ref                                                                  title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
haseebindata/student-performance-predictions                         Student Performance Predictions                      9KB  2024-08-17 06:57:57           3618         83  0.9411765        
mrmars1010/filpkart-mobiles                                          Flipkart Products Sales: Mobiles Dataset 📱         108KB  2024-08-24 04:31:50           1081         25  0.88235295       
berkayalan/paris-2024-olympics-medals                                Paris 2024 Olympics Medals                           1KB  2024-08-14 11:02:45           3320         63  1.0              
abdullahashfaqvirk/student-mental-health

In [8]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 43% 11.0M/25.7M [00:00<00:00, 111MB/s]
100% 25.7M/25.7M [00:00<00:00, 153MB/s]


In [9]:
!unzip imdb-dataset-of-50k-movie-reviews.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [10]:
import numpy as np
import pandas as pd

In [65]:
df = pd.read_csv('IMDB Dataset.csv')

In [66]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [67]:
df = df.sample(10000)

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 25298 to 20915
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     10000 non-null  object
 1   sentiment  10000 non-null  object
dtypes: object(2)
memory usage: 234.4+ KB


In [69]:
df['sentiment'].replace({'positive':1,'negative':0},inplace = True)

In [70]:
df.head()

,review,sentiment
25298,Saw this piece of work at a film fest in CA. M...,0
39983,I cannot believe I sat through this utter wast...,0
11892,***SPOILERS*** Seething with hatred and reveng...,1
38197,"The word ""boring"" gets thrown around way too o...",0
18856,I can fondly remember Bo Derek's heyday and th...,0


In [71]:
import re

In [72]:
def html_clean(text):
  clean = re.compile('<.*?>')
  return re.sub(clean,'',text)

In [73]:
df['review'] = df['review'].apply(html_clean)

In [74]:
df.head()

,review,sentiment
25298,Saw this piece of work at a film fest in CA. M...,0
39983,I cannot believe I sat through this utter wast...,0
11892,***SPOILERS*** Seething with hatred and reveng...,1
38197,"The word ""boring"" gets thrown around way too o...",0
18856,I can fondly remember Bo Derek's heyday and th...,0


In [75]:
def con_lower(text):
  return text.lower()

In [76]:
df['review'] = df['review'].apply(con_lower)

In [77]:
df.head()

,review,sentiment
25298,saw this piece of work at a film fest in ca. m...,0
39983,i cannot believe i sat through this utter wast...,0
11892,***spoilers*** seething with hatred and reveng...,1
38197,"the word ""boring"" gets thrown around way too o...",0
18856,i can fondly remember bo derek's heyday and th...,0


In [78]:
def remove_special(text):
  x=''
  for i in text:
    if i.isalnum():
      x=x+i
    else:
      x = x+' '
  return x

In [79]:
df['review'] = df['review'].apply(remove_special)

In [80]:
df['review'].head()

,review
25298,saw this piece of work at a film fest in ca m...
39983,i cannot believe i sat through this utter wast...
11892,spoilers seething with hatred and reveng...
38197,the word boring gets thrown around way too o...
18856,i can fondly remember bo derek s heyday and th...


In [86]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [87]:
from nltk.corpus import stopwords

In [88]:
def remove_stopwords(text):
  x = []
  for i in text.split():
    if i not in stopwords.words('english'):
      x.append(i)
  y = x[:]
  x.clear()
  return y

In [89]:
df['review'] = df['review'].apply(remove_stopwords)

In [90]:
df.head()

,review,sentiment
25298,"[saw, piece, work, film, fest, ca, god, direct...",0
39983,"[cannot, believe, sat, utter, waste, time, fas...",0
11892,"[spoilers, seething, hatred, revenge, half, br...",1
38197,"[word, boring, gets, thrown, around, way, ofte...",0
18856,"[fondly, remember, bo, derek, heyday, uk, pres...",0


In [91]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [92]:
y = []
def stem_words(text):
  for i in text:
    y.append(ps.stem(i))
  z = y[:]
  y.clear()
  return z

In [93]:
df['review'] = df['review'].apply(stem_words)

In [94]:
def join_back(text):
  return " ".join(text)

In [95]:
df['review'] = df['review'].apply(join_back)

In [97]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [98]:
x = cv.fit_transform(df['review']).toarray()

In [99]:
y = df.iloc[:,-1].values

In [100]:
y.shape

(10000,)

In [101]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2)

In [102]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB

In [103]:
clf1 = GaussianNB()
clf2 = MultinomialNB()

clf1.fit_transform(x_train,y_train)
clf2.fit_transform(x_train,y_train)

In [105]:
clf1.fit(x_train,y_train)
clf2.fit(x_train,y_train)

MultinomialNB()

In [106]:
y_pred1 = clf1.predict(x_test)
y_pred2 = clf2.predict(x_test)

In [107]:
from sklearn.metrics import accuracy_score

In [109]:
print("Gaussian ", accuracy_score(y_test,y_pred1))
print("Mulitnomial ", accuracy_score(y_test,y_pred2))

Gaussian  0.5985
Mulitnomial  0.8325
